<a href="https://colab.research.google.com/github/dimitrod/ehu_nlp_dimathina/blob/Evaluation/Evaluation_Chain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
!git clone https://github.com/mandarjoshi90/triviaqa.git

fatal: destination path 'triviaqa' already exists and is not an empty directory.


In [37]:
%cd triviaqa

/content/triviaqa


In [38]:
!pip install -r requirements.txt

In [39]:
import subprocess
import os
import json
from tqdm import tqdm
import importlib
import argparse

In [40]:
triviaqa_root = "/content/triviaqa"

In [41]:
env = os.environ.copy()
env["PYTHONPATH"] = triviaqa_root

In [42]:
%cd ..

/content


In [43]:
!pip install datasets transformers huggingface_hub

In [44]:
def evaluate_triviaqa(dataset_file, prediction_file):
  script_path = "/content/triviaqa/evaluation/triviaqa_evaluation.py"
  result = subprocess.run([
    "python", script_path,
    "--dataset_file", dataset_file,
    "--prediction_file", prediction_file],
    env=env,
    capture_output=True,
    text=True,
  )
  print(result.stdout)

In [45]:
def get_answers(questions, model):
    answers = {}

    for i in tqdm(range(len(questions)), desc="Retrieving Answers"):
        question = questions[i]
        answers[question[0]] = model.invoke(question[1])
    return answers

In [46]:
def load_questions(path, split):
    module = importlib.import_module(path+".load_evaluation_data")
    Data_Loader = getattr(module, "data_loader")
    loader = Data_Loader()
    loader.load_evaluation_set(split)

    with open(f"evaluation_data.json", "r", encoding="utf-8") as f:
        data = json.loads(f.read())

    questions = []

    for i in tqdm(range(split),desc="Loading Questions"):
        question = data["Data"][i]
        questions.append([question["QuestionId"], question["Question"]])
    return questions

In [47]:
def load_model(path, model_name):
  module = importlib.import_module(path)
  model = getattr(module, model_name)
  print("Loading model...")
  return model()

In [48]:
def get_args():
    parser = argparse.ArgumentParser(
        description='Evaluation Chain for TriviaQA {}')
    parser.add_argument('--dataset_file', help='Dataset file')
    parser.add_argument('--model_file', help='Model file')
    args = parser.parse_args()
    return args

In [50]:
if __name__ == "__main__":
    #args = get_args()
    directory = "evaluation_data"
    split = 10
    predictions_path = "/content/evaluation_data/predictions.json"
    model_path = "model.tiny_llama"
    model_name = "tinyllama"


    questions = load_questions(directory, split)
    model = load_model(model_path, model_name)
    print("Model initiated")
    print("Starting QA...")
    answers = get_answers(questions, model)
    with open(predictions_path, "w", encoding="utf-8") as f:
        f.write(json.dumps(answers))
    print("Test results:")
    evaluate_triviaqa("/content/"+directory+"/evaluation_data.json", predictions_path)


TypeError: data_loader.load_evaluation_set() missing 1 required positional argument: 'directory'